In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from models import Transformer

In [2]:
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
train = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del train['lic']
train.tail()

,src,tar
232731,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
232732,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
232733,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
232734,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...
232735,I went drinking with one of my boyfriend's fri...,« Je suis allée boire avec un ami de mon compa...


In [4]:
train.tail(1)['src'].values

array(['I went drinking with one of my boyfriend\'s friends, and now he\'s furious at me. "Was this friend a guy or a girl?" "A guy, obviously. Why would I go drinking with his female friends?" "Yeah, you\'re right." "His name is Tom. He\'s really hot, and I really want to go drinking with him again."'],
      dtype=object)

In [5]:
# 정규표현식 활용하여 데이터 전처리
def clean_text(inputString):
    text_rmv = re.sub(r'[\\\xa0·«»]', '', inputString)
    # 다수 개의 공백을 하나의 공백으로 치환
    text_rmv = re.sub(r"\s+", " ", text_rmv)
    return text_rmv

In [6]:
train['tar'] = train['tar'].apply(lambda x: clean_text(x))
train['src'] = train['src'].apply(lambda x: clean_text(x))
train

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
232731,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
232732,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
232733,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
232734,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [7]:
total_len = len(train)
total_len

232736

In [8]:
idx = np.random.permutation(total_len)
idx

array([ 19895, 198409,  78819, ..., 181329,  58368, 109541])

In [9]:
int(total_len*0.8)

186188

In [10]:
train_df, test_df = train.iloc[idx[:int(total_len*0.9)]], train.iloc[idx[int(total_len*0.9):]]

In [11]:
train_df.reset_index(inplace=True, drop=True)
train_df

,src,tar
0,Give me my drink.,Donnez-moi mon verre !
1,Can you tell me what you're talking about?,Est-ce que tu peux me dire de quoi tu parles?
2,Tom's wallet was stolen.,Le portefeuille de Tom a été volé.
3,I love this city.,J'adore cette ville.
4,I've left Tom three messages.,J'ai laissé trois messages à Tom.
...,...,...
209457,You will always be welcome.,Vous serez toujours bienvenu.
209458,I wonder how many people saw Tom doing that.,Je me demande combien de personnes ont vu Tom ...
209459,What's the name of your insurance company?,Quel est le nom de ta compagnie d'assurance ?
209460,"I do not like tea, so I generally drink coffee...","Je n'aime pas le thé, donc je bois généralemen..."


In [12]:
test_df.reset_index(inplace=True, drop=True)
test_df

,src,tar
0,"As long as I know the money is safe, I will no...","Pour autant que je sache, l'argent est en sécu..."
1,You sound tired.,Vous paraissez fatigué.
2,My career is on the line.,Ma carrière est en jeu.
3,Tom saw Mary waiting at the bus stop.,Tom vit Mary attendre à l'arrêt de bus.
4,Do you always help Tom with his homework?,Aides-tu toujours Tom à faire ses devoirs ?
...,...,...
23269,Where can I park the car?,Où est-ce que je peux garer la voiture?
23270,She does know it.,Elle sait certainement cela.
23271,You've touched on an important issue.,Vous avez abordé une question importante.
23272,I'll be out in a jiff.,Je serai dehors en moins de deux.


In [13]:
train.tail(1)['src'].values[0]

'I went drinking with one of my boyfriend\'s friends, and now he\'s furious at me. "Was this friend a guy or a girl?" "A guy, obviously. Why would I go drinking with his female friends?" "Yeah, you\'re right." "His name is Tom. He\'s really hot, and I really want to go drinking with him again."'

In [14]:
train.tail(1)['tar'].values[0]

"Je suis allée boire avec un ami de mon compagnon, et voilà qu'il est furieux contre moi. Était-ce un gars ou une fille? Un gars, bien évidemment. Pourquoi irais-je boire avec ses amies? Ouais, ça se comprend. Il s'appelle Tom. Il est trop canon, et j'ai tellement envie d'aller prendre un verre avec lui à nouveau."

In [15]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [16]:
print('영어 최대 길이: ', np.max(train['src'].apply(lambda x: len(tokenizer.tokenize(x)))))
print('불어 최대 길이: ', np.max(train['tar'].apply(lambda x: len(tokenizer.tokenize(x)))))

영어 최대 길이:  88
불어 최대 길이:  112


In [17]:
tokenizer.add_special_tokens({'bos_token': '<s>'})
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.pad_token)

<s>
</s>
<pad>


In [18]:
tokenizer.vocab_size

32100

In [19]:
tokenizer.decode(0)

'<pad>'

In [20]:
print(tokenizer.pad_token_id)

0


In [21]:
max_len = 120

In [22]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.src = self.tokenizer(list(self.data['src']), padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        self.tar = self.tokenizer(['<s>' + s for s in self.data['tar']], padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.src[idx], self.tar[idx]

In [23]:
custom_ds = CustomDataset(train, tokenizer, max_len)
train_ds, valid_ds, test_ds = torch.utils.data.random_split(custom_ds, [0.8, 0.1, 0.1])
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True)

In [24]:
len(train_dl)

2910

In [25]:
i = 6
idx = train_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

121766
tensor([  27,   31,   51,  182, 2764,   13,   66,   13,   25,    5,    1,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
I'm very proud of all of you.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [26]:
i = 6
idx = valid_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

37391
tensor([3059, 9460,    7,   12,    3, 1544,    5,    1,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
Tom refuses to eat.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [27]:
i = 6
idx = test_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

54499
tensor([ 148, 2124,    6,  737,   31,   17,   25,   58,    1,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
You knew, didn't you?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [28]:
src_texts, tar_texts = next(iter(train_dl))
print(src_texts.shape)
print(src_texts[0])
print(tokenizer.decode(src_texts[0]))
print(tar_texts.shape)
print(tar_texts[0])
print(tokenizer.decode(tar_texts[0]))

torch.Size([64, 89])
tensor([ 328,   31,   60, 1556,   69, 2324,    5,    1,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
They're playing our song.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [29]:
vocab_size = tokenizer.vocab_size + 1
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 120
dropout = 0.1
batch_size = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

model = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, batch_size, device).to(device)

In [30]:
print(model)

Transformer(
  (encoder_embedding): InputEmbeddings(
    (embedding): Embedding(32101, 512)
  )
  (decoder_embedding): InputEmbeddings(
    (embedding): Embedding(32101, 512)
  )
  (enc_positional_embedding): Embedding(120, 512)
  (dec_positional_embedding): Embedding(120, 512)
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttentionLayer(
        (W_q): Linear(in_features=512, out_features=512, bias=False)
        (W_k): Linear(in_features=512, out_features=512, bias=False)
        (W_v): Linear(in_features=512, out_features=512, bias=False)
        (W_o): Linear(in_features=512, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): PositionwiseFeedForward(
        (fc_1): Linear(in_features=512, out_features=2048, bias=True)
        (fc_2): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((512,), eps=1e

In [31]:
src_texts = src_texts.to(device)
tar_texts = tar_texts.to(device)
output, attention = model(src_texts, tar_texts[:, :-1])

tensor([[[  6.6093,  -3.9997, -28.5978,  ...,  -8.3081, -13.6138,  41.8559],
         [ 20.4056,  -2.5965,  12.5584,  ...,  -5.6384, -10.6591,  -9.5875],
         [-20.2084,  -0.4611, -29.0431,  ...,  67.5148, -23.4499,  30.1889],
         ...,
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946],
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946],
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946]],

        [[-27.8549, -19.3981,  58.7812,  ...,  -6.7235,  12.5520,  49.9758],
         [ 20.4056,  -2.5965,  12.5584,  ...,  -5.6384, -10.6591,  -9.5875],
         [ 12.0666,   1.9082, -34.3673,  ...,  -1.5215,  35.2356, -17.2235],
         ...,
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946],
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946],
         [-23.8635, -13.8896, -24.2837,  ...,   3.2488,  11.0449,  -1.8946]],

        [[-15.3123, -31.4079, -43.2735,  ...

AttributeError: 'Transformer' object has no attribute 'self'

In [ ]:
model.forward

In [32]:
src_texts.shape

torch.Size([64, 89])

In [33]:
tar_texts.shape

torch.Size([64, 114])

In [34]:
output.shape

torch.Size([64, 113, 32101])

In [35]:
attention.shape

torch.Size([64, 8, 113, 89])

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
tokenizer.add_special_tokens({'bos_token':'<s>'})

1

In [7]:
src_tensor = tokenizer(['That was pathetic'], padding=True, truncation=True, max_length = 120, return_tensors='pt').input_ids[0].unsqueeze(0)

In [10]:
src_tensor.unsqueeze(0).shape

torch.Size([1, 1, 5])

In [9]:
src_tensor.shape

torch.Size([1, 5])

In [13]:
src_tensor.shape

torch.Size([1, 5])

In [14]:
model.make_src_mask(src_tensor)

tensor([[[[True, True, True, True, True]]]])

In [15]:
src_mask = model.make_src_mask(src_tensor)
enc_output = src_tensor
with torch.no_grad():
    for enc_layer in model.encoder_layers:
        enc_output = enc_layer(enc_output, src_mask)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_mm)